# Db2Sharp

This db2sharp application demonstrates the acceleration of the databroker interface for accessing and preprocessing large ptychographic datasets, for example: loading files directly within the script (106 s) vs loading files with the Spark parallel platform (22 s). 

In [1]:
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
import h5py

# make graphics inline
%matplotlib inline

from SparkSharpReader import SparkSharpReader
from SharpWriter import SharpWriter

# Defining Experiment-Specific Parameters

In [2]:
sid = 27223
fields = ['zpssx','zpssy']

sharpReader = SparkSharpReader()
# x_c, y_c, xn, yn, threshold
sharpReader.init(64, 147, 128, 128, 2)

sharpWriter = SharpWriter()
# pixel size (um), distance (m), wavelength (nm), det_side 
sharpWriter.init(55, 0.64, 0.1331, 128) 

prbfile = '../../data/27223/recon_21678_t4_probe_ave_rp.npy'

cxifile = '../../data/27223/hxn27223.cxi'

# Getting Metadata from Databroker

In [3]:
print("getting fnames, points from db ...");
t1 = datetime.now();
fnames, ic = sharpReader.get_merlin1_fnames(sid)
xs, ys = sharpReader.get_points(sid, fields)
t2 = datetime.now();
print ("processing time: ", (t2 - t1), ", fnames: ", len(fnames), ", ic: ", len(ic), ", x: ", len(xs), ", y: ", len(ys));

getting fnames, points from db ...
processing time:  0:00:18.517880 , fnames:  67200 , ic:  67200 , x:  67200 , y:  67200


# Loading Files Directly within this Script

In [4]:
print("loading files ...");
t1 = datetime.now();
frames = sharpReader.load_files(sid, fnames, ic)
t2 = datetime.now();
print ("processing time: ", (t2 - t1), "frames: ", len(frames));

loading files ...
processing time:  0:01:46.392514 frames:  67200


# Loading Files with Spark Workers

In [5]:
partitions = 20
print("loading files ...");
t1 = datetime.now();
frames = sharpReader.load_files_with_spark(sid, fnames, ic, partitions)
t2 = datetime.now();
print ("processing time: ", (t2 - t1), "frames: ", len(frames));

loading files ...
processing time:  0:00:21.825956 frames:  67200


# Writing Data to the SHARP-NSLS2 Input File 

In [6]:
print("write a cxi file ...");
t1 = datetime.now();
sharpWriter.write(cxifile, prbfile, frames, xs, ys)
t2 = datetime.now();
print ("processing time: ", (t2 - t1))

write a cxi file ...
processing time:  0:00:11.500009
